In [1]:
#%matplotlib inline
from PIL import Image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from skimage.filters import threshold_otsu
from skimage.transform import resize
from skimage.transform import rotate
from skimage.util import view_as_windows
from collections import defaultdict
import cv2

In [2]:
np.random.seed(1234)

In [5]:
df = pd.read_csv('mass_case_description_test_set.csv')
df.head()

,patient_id,breast_density,left or right breast,image view,abnormality id,abnormality type,mass shape,mass margins,assessment,pathology,subtlety,image file path,cropped image file path,ROI mask file path
0,P_00016,4,LEFT,CC,1,mass,IRREGULAR,SPICULATED,5,MALIGNANT,5,Mass-Test_P_00016_LEFT_CC/1.3.6.1.4.1.9590.100...,Mass-Test_P_00016_LEFT_CC_1/1.3.6.1.4.1.9590.1...,Mass-Test_P_00016_LEFT_CC_1/1.3.6.1.4.1.9590.1...
1,P_00016,4,LEFT,MLO,1,mass,IRREGULAR,SPICULATED,5,MALIGNANT,5,Mass-Test_P_00016_LEFT_MLO/1.3.6.1.4.1.9590.10...,Mass-Test_P_00016_LEFT_MLO_1/1.3.6.1.4.1.9590....,Mass-Test_P_00016_LEFT_MLO_1/1.3.6.1.4.1.9590....
2,P_00017,2,LEFT,CC,1,mass,ROUND,CIRCUMSCRIBED,4,MALIGNANT,4,Mass-Test_P_00017_LEFT_CC/1.3.6.1.4.1.9590.100...,Mass-Test_P_00017_LEFT_CC_1/1.3.6.1.4.1.9590.1...,Mass-Test_P_00017_LEFT_CC_1/1.3.6.1.4.1.9590.1...
3,P_00017,2,LEFT,MLO,1,mass,ROUND,ILL_DEFINED,4,MALIGNANT,4,Mass-Test_P_00017_LEFT_MLO/1.3.6.1.4.1.9590.10...,Mass-Test_P_00017_LEFT_MLO_1/1.3.6.1.4.1.9590....,Mass-Test_P_00017_LEFT_MLO_1/1.3.6.1.4.1.9590....
4,P_00032,3,RIGHT,CC,1,mass,ROUND,OBSCURED,0,BENIGN,2,Mass-Test_P_00032_RIGHT_CC/1.3.6.1.4.1.9590.10...,Mass-Test_P_00032_RIGHT_CC_1/1.3.6.1.4.1.9590....,Mass-Test_P_00032_RIGHT_CC_1/1.3.6.1.4.1.9590....


In [6]:
CALC_TARGET_RESIZE = np.array([2750,1500])
MASS_TARGET_RESIZE = np.array([1100, 600])
MAX_ROTATE = 30 ## degrees
STEP_SIZE = 100 ## Stride for getting windows

MASK_CUTOFF = 0 ## If a patch has an average mask value of 0 discard it as it is not in the breast
ROI_CUTOFF = 0 

In [7]:
###################################################################
# Read in Files
###################################################################
def get_im_as_array(file_name, file_type):
    '''
    Read in an image and yield it as a numpy array
    params: 
        file_name: name of the file
        file_type: either 'full' or 'ROI'
    
    '''
    if file_type == 'full':
        path = PATH_TO_FILES
    elif file_type == 'ROI': ## ROI
        path = PATH_TO_ROI
    else: 
        print("Enter file type as either 'full' or 'ROI'")
        pass

    file_path = os.path.join(path,file_name)
    im = Image.open(file_path)
    return np.asarray(im)

In [8]:
###################################################################
# Associate images with labels
###################################################################

def get_labels(path_to_csv):
    '''
    Concatenates various components of the named files to a list and returns the file_name and pathology
    params:
        path_to_csv: path to the CSV with the file list
    returns: 
        a data frame containing the file_name (as an index) and the pathology.  
    '''
    df = pd.read_csv(path_to_csv)
    df['file_name'] = 'Calc-Training_' + df['patient_id'] + '_' + df['side'] + '_' + df['view'] + '_' + df['abn_num'].astype(str) + '_mask.png'
    df = df[['file_name', 'pathology']]
    df.set_index('file_name', inplace=True)
    return df

def get_mask_list():
    '''
    Associate each file with all of its masses and their pathology (benign, malignant, other).
    Return a dictionary of {file_name: (mask, pathology)}
    '''
    mask_list = defaultdict(list)
    roi_files = os.listdir(PATH_TO_ROI)
    df = get_labels(PATH_TO_ROI_CSV_LABELS)

    for file_name in roi_files:
        mask_list[file_name[:-11]].append((file_name, df.loc[file_name]['pathology']))
    
    return mask_list
